In [1]:
import os
from langchain_community.chat_models import ChatOllama
from langgraph.graph import END, StateGraph
from typing import Dict, TypedDict
from langchain.prompts import PromptTemplate

In [2]:
local_llm = 'elvee/hermes-2-pro-llama-3:8b-Q5_K_M'
llm = ChatOllama(model=local_llm, temperature=0)

Define tool

In [3]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def web_search(query:str) -> str:
    """Search the web for query"""
    tool = TavilySearchResults(max_results=3,tavily_api_key=os.environ['TAVILY_API_KEY'])
    return str(tool.invoke(query))

In [4]:
print(web_search.name)
print(web_search.description)
print(web_search.args)

web_search
Search the web for query
{'query': {'title': 'Query', 'type': 'string'}}


Render tool description which prompts the llm

In [5]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([web_search])
rendered_tools

'web_search(query: str) -> str - Search the web for query'

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool: 

""" + rendered_tools + """

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.
For example: 
{{'name': 'web_search', 'arguments': {{'arg1': "value_1"}}}}


"""

print(system_prompt)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool: 

web_search(query: str) -> str - Search the web for query

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.
For example: 
{{'name': 'web_search', 'arguments': {{'arg1': "value_1"}}}}





In [7]:
from langchain_core.output_parsers import JsonOutputParser


chain = prompt | llm | JsonOutputParser()
chain.invoke({"input": "Search Lawrence Wong"})

{'name': 'web_search', 'arguments': {'query': 'Lawrence Wong'}}

In [8]:
from operator import itemgetter

def search(query):
    search_chain = prompt | llm | JsonOutputParser() | itemgetter("arguments") | web_search
    return search_chain.invoke({"input": query})

search("Lawrence Wong")


KeyError: 'TAVILY_API_KEY'

In [ ]:
summariser_system_prompt = """

The JSON object you are given contains some search results. 
Return a summary of the search results. 

"""

def search_summarise(query):
    search_result = search(query)
    summariser_prompt = ChatPromptTemplate.from_messages(
        [("system", summariser_system_prompt + search_result.replace("{", "[").replace("}", "]"))]
    )
    summariser_chain = summariser_prompt | llm 
    return summariser_chain.invoke({}).content

search_summarise("Speakers for ATX Summit")

AttributeError: 'AIMessage' object has no attribute 'replace'